In [1]:
from dask.distributed import Client
import dask.dataframe as df

client = Client(processes=False, threads_per_worker=2,
                n_workers=3, memory_limit='4GB')
client

/Users/alexiuk/projects/baseline0/-Scalable-Data-Analysis-in-Python-with-Dask-Kashif/venv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61980 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.0.0.227:61980/status,
Dashboard: http://10.0.0.227:61980/status,Workers: 3
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://10.0.0.227/28234/1,Workers: 3
Dashboard: http://10.0.0.227:61980/status,Total threads: 6
Started: Just now,Total memory: 11.18 GiB
Comm: inproc://10.0.0.227/28234/4,Total threads: 2
Dashboard: http://10.0.0.227:61981/status,Memory: 3.73 GiB
Nanny: None,


In [2]:
import time

def some_func(x):
    time.sleep(1)
    y = x + 1
    time.sleep(1)
    z = x * y
    time.sleep(1)
    return z

## `Client.submit` returns a Future Object

In [3]:
result_1 = client.submit(some_func, 5)  

In [4]:
result_1

<Future: pending, key: some_func-a4bf80455161d81709dce00a327dbc57>

## In this case the computation will be pending

In [5]:
def some_func_long(x):
    time.sleep(10)
    y = x + 1
    time.sleep(5)
    z = x * y
    time.sleep(2)
    return z

In [6]:
result_2 = client.submit(some_func_long, 5)  

In [7]:
result_2

<Future: pending, key: some_func_long-f9d8fa0e328fe9d1f7d879d60afe588f>

## `result()` function can be used to force the computation to complete

In [8]:
result_1.result()

30

In [9]:
result_3 = client.submit(some_func_long, 15)  

In [10]:
result_3.result()

240

## We can evaluate multiple Future objects in one go using `gather`

In [11]:
futures_list = [client.submit(some_func, x)
               for x in range(1, 8)]

In [12]:
all_res = client.gather(futures_list)

In [13]:
all_res

[2, 6, 12, 20, 30, 42, 56]

## We can also use `map` function to pass multiple future object

In [14]:
list_result = client.map(some_func_long, range(5))

In [15]:
client.gather(list_result)

[0, 2, 6, 12, 20]

### Reference : http://docs.dask.org/en/latest/futures.html